## Running prompts on OpenAI

In [1]:
#%pip install markdown
#%pip install annoy
#%pip install openai

In [2]:
from openai import OpenAI # OpenAI API
import json
import requests # to download some resources
import os # file operations
import numpy as np # linear algebra
import pandas as pd # data processing
from markdown import markdown # to render markdown
from IPython.display import Markdown
import annoy # Approximate Nearest Neighbors Oh Yeah for fast searching
import pickle
from annoy import AnnoyIndex

In [3]:
# read in our api key
with open('../api-keys/our_api_key.txt', 'r') as file:
    api_key = file.read().replace('\n', '')

# read in finns api key (the one we'll use for testing)
with open('../api-keys/finns_api_key.txt', 'r') as file:
    finns_api_key = file.read().replace('\n', '')

In [4]:
# Setting the OpenAI
client = OpenAI(api_key=finns_api_key) # using finns for testing

#### Sample of countries

In [5]:
# Sample of countries
sample_countries = ['Finland','Netherlands','Denmark','Norway', 'Chile', 'Bangladesh']

# Folder with raw PDFs
folder_mds = "../data/3-naps-md"

# Importing MDs files
for file in os.listdir(folder_mds):
    for country in sample_countries:
        if country in file:  
            with open(os.path.join(folder_mds, file), "r", encoding="utf-8") as md_file:
                content = md_file.read()
            globals()[country.lower()] = content # Saving the MD file in lowercase


### 1. Using the WHOLE document

In [6]:
country_test = norway
country_name = "norway"

In [7]:
system_prompt = """
You are an assistant that extracts and summarizes national action plans for antimicrobial resistance (NAPs). Your task is to answer the following questions concisely while relying strictly on the document content. 

For each question, provide:
1. A 'yes' or 'no' answer.
2. The specific text from the document that supports the answer.

Here are the questions:

1. What is the period considered for the NAP? (Provide the exact years or date range)
2. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Human Health"? (yes/no + supporting text)
3. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Animal Health"? (yes/no + supporting text)
4. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Environment"? (yes/no + supporting text)
5. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Agriculture/Food Security"? (yes/no + supporting text)
6. Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools? (yes/no + supporting text)
7. Does the NAP include any specific, time-bound targets to track progress? (yes/no + supporting text)
8. Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR? (yes/no + supporting text)
9. Does the NAP establish a multisectoral committee or task force for AMR coordination? (yes/no + supporting text)
10. Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists? (yes/no + supporting text)
11. Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement? (yes/no + supporting text)
12. Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE? (yes/no + supporting text)
13. Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions? (yes/no + supporting text)
14. Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)? (yes/no + supporting text)
15. Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control? (yes/no + supporting text)
16. Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance? (yes/no + supporting text)
17. Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics? (yes/no + supporting text)
18. Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use? (yes/no + supporting text)
19. Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws? (yes/no + supporting text)
20. Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines? (yes/no + supporting text)

Format the response as a JSON object:
{
  "period_start": "XXXX",
  "period_end":"YYYY",
  "questions": [
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?",
      "answer": "yes",
      "supporting_text": "The NAP mentions specific actions in the human health sector, including..."
    },
    ...
  ]
}
"""

In [8]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,  # Ensures deterministic output
    top_p=0.5,  # Controls diversity
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": country_test}
    ]
)

# Print the structured JSON response
print(response.choices[0].message.content)

{
  "period_start": "2015",
  "period_end": "2020",
  "questions": [
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?",
      "answer": "yes",
      "supporting_text": "The NAP includes specific goals for human health, such as reducing antibiotic use in the total inhabitants by 30 percent."
    },
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?",
      "answer": "yes",
      "supporting_text": "The NAP outlines goals for food-producing animals, including reducing antibiotic use in terrestrial animals by at least 10 percent."
    },
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?",
      "answer": "yes",
      "supporting_text": "The NAP includes goals for mapping antibiotic resistant bacteria in representative environments and studying the effects of 

In [9]:
# Formatting the JSON

c = json.loads(response.choices[0].message.content)
period_data = pd.DataFrame([
    {"question": "period_start", "answer": c["period_start"], "supporting_text": None},
    {"question": "period_end", "answer": c["period_end"], "supporting_text": None}
])

df = pd.DataFrame(c["questions"])
df = pd.concat([period_data, df], ignore_index=True)
df['country']=country_name
df = df.rename(columns={"question": "question_text", "answer": "answer_full"})
df['question']=df['question_text']
df.loc[2:, 'question'] = range(1, 1 + len(df) - 2)
df = df.drop(columns=['question_text'])
output_json = df.to_dict(orient="records")

# Saving the JSON
output_path = f"../data/4-summary-responses-json/version_yesno_original/summary_response_{country_name}.json"
with open(output_path, "w") as f:
    json.dump(output_json, f, indent=4)

print(f"Saved response to {output_path}")


Saved response to ../data/4-summary-responses-json/summary_response_norway.json


In [10]:
df.head()

,answer_full,supporting_text,country,question
0,2015,None,norway,period_start
1,2020,None,norway,period_end
2,yes,The NAP includes specific goals for human heal...,norway,1
3,yes,The NAP outlines goals for food-producing anim...,norway,2
4,yes,The NAP includes goals for mapping antibiotic ...,norway,3


In [11]:
# Load the annotations
annotations_df = pd.read_excel('../data/5-annotations/annotations.xlsx')
annotations_df['question'] = annotations_df['question'].astype(str)
annotations_df['answer'] = annotations_df['answer'].astype(str)

# Filter for the current country
annotations_df = annotations_df[annotations_df['country'] == country_name]


In [12]:
annotations_df

,country,question,answer,comment_input_for_protocol,supporting_text_when_true_answer_is_yes_but_prediction_is_no
21,norway,period_start,2015,NaN,NaN
22,norway,period_end,2020,NaN,NaN
23,norway,1,yes,NaN,NaN
24,norway,2,yes,NaN,NaN
25,norway,3,yes,NaN,NaN
26,norway,4,yes,NaN,NaN
27,norway,5,yes,NaN,NaN
28,norway,6,yes,NaN,NaN
29,norway,7,yes,NaN,NaN
30,norway,8,yes,NaN,NaN


In [13]:
# Extract period and questions from the model response
model_answers = [{"country": country_name, "question": "period_start", "answer": c["period_start"]}]  # Corrected
model_answers.append({"country": country_name, "question": "period_end", "answer": c["period_end"]})  # Corrected

# Add answers for the other questions
for q in c["questions"]:
    model_answers.append({
        "country": country_name,
        "question": q["question"],
        "answer": q["answer"]
    })

# Convert to DataFrame
model_answers_df = pd.DataFrame(model_answers)
model_answers_df['question'] = model_answers_df['question'].astype(str)

# Mapping the questions to numbers
question_mapping = {
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?": "1",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?": "2",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?": "3",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Agriculture/Food Security'?": "4",
    "Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools?": "5",
    "Does the NAP include any specific, time-bound targets to track progress?": "6",
    "Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR?": "7",
    "Does the NAP establish a multisectoral committee or task force for AMR coordination?": "8",
    "Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists?": "9",
    "Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement?": "10",
    "Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE?": "11",
    "Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions?": "12",
    "Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)?": "13",
    "Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control?": "14",
    "Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance?": "15",
    "Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics?": "16",
    "Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use?": "17",
    "Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws?": "18",
    "Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines?": "19"
}

# Replacing the questions with numbers
model_answers_df["question"] = model_answers_df["question"].replace(question_mapping)

# Merge based on country and question
merged_df = pd.merge(annotations_df, model_answers_df, on=["country", "question"], suffixes=('_gs', '_llm'))

# Now you should have the full merged dataframe


In [14]:
merged_df

,country,question,answer_gs,comment_input_for_protocol,supporting_text_when_true_answer_is_yes_but_prediction_is_no,answer_llm
0,norway,period_start,2015,NaN,NaN,2015
1,norway,period_end,2020,NaN,NaN,2020
2,norway,1,yes,NaN,NaN,yes
3,norway,2,yes,NaN,NaN,yes
4,norway,3,yes,NaN,NaN,yes
5,norway,4,yes,NaN,NaN,yes
6,norway,5,yes,NaN,NaN,yes
7,norway,6,yes,NaN,NaN,yes
8,norway,7,yes,NaN,NaN,no
9,norway,8,yes,NaN,NaN,yes


In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert categorical yes/no answers to binary (1 for yes, 0 for no)
merged_df['gs_binary'] = merged_df['answer_gs'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))
merged_df['llm_binary'] = merged_df['answer_llm'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))

# Drop rows with NaN values in either gs_binary or llm_binary for accurate calculations
yesno_df = merged_df[~merged_df['question'].isin(["period_start", "period_end"])]
yesno_df = yesno_df.dropna(subset=['gs_binary', 'llm_binary'])

# Performance for yes/no questions
accuracy = accuracy_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
precision = precision_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
recall = recall_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
f1 = f1_score(yesno_df['gs_binary'], yesno_df['llm_binary'])

# Performance for period extraction (period_start, period_end)
period_matches = merged_df[merged_df['question'].isin(["period_start", "period_end"])]

# For period extraction, we don't need binary conversion, so we directly compare the answers
period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']

# Calculate period extraction accuracy as the percentage of exact matches
period_accuracy = period_matches['period_match'].mean()  # Exact match percentage

# Print results
print(f"Accuracy (Yes/No Questions): {accuracy:.2f}")
print(f"Precision (Yes/No Questions): {precision:.2f}")
print(f"Recall (Yes/No Questions): {recall:.2f}")
print(f"F1 Score (Yes/No Questions): {f1:.2f}")
print(f"Period Extraction Accuracy: {period_accuracy:.2f}")

Accuracy (Yes/No Questions): 0.84
Precision (Yes/No Questions): 0.94
Recall (Yes/No Questions): 0.89
F1 Score (Yes/No Questions): 0.91
Period Extraction Accuracy: 1.00


/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gq/T/ipykernel_85981/2876663377.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']


---